In [1]:
import pandas as pd
import plotly.express as px
import sqlite3

from jupyter_dash import JupyterDash
from dash import dcc, html
from dash import html
import plotly.graph_objects as go

from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(r".\db_motogp.sqlite")

race_results_view = pd.read_sql_query("SELECT * from race_results_view", con)
rider_stats = pd.read_sql_query("SELECT * from rider_stats_view", con)
race_condition = pd.read_sql_query("SELECT * from race_conditions WHERE category = 'MotoGP' ", con)
countries = pd.read_sql_query("SELECT * from countries",con)
circuits = pd.read_sql_query("SELECT * FROM circuits",con)
con.close()


### Riders performance thru specific season

In [2]:
riders_2021 = race_results_view.loc[(race_results_view['year'] == 2021) & (race_results_view['category'] == 'MotoGP')]
points_by_race_and_rider = riders_2021.groupby(['sequence', 'rider_name'])['points'].sum().reset_index().sort_values('rider_name')
dummy_points = pd.DataFrame({'rider_name': points_by_race_and_rider['rider_name'].unique(), 'sequence': 0, 'points': 0})
points_by_race_and_rider = pd.concat([dummy_points, points_by_race_and_rider], ignore_index=True)
grouped_points = points_by_race_and_rider.sort_values('sequence').groupby('rider_name')['points'].cumsum()
points_by_race_and_rider['cumulative_points'] = grouped_points

#only to have top 3 riders:
points_by_rider_sum = riders_2021.groupby('rider_name')['points'].sum().reset_index().sort_values('points', ascending = False)
top_5_riders = points_by_rider_sum.head(3)

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id = "filter",
        options = [],
        value =[],
        multi = True
    ),
    dcc.Slider(
    id = "year_slider",
    min = 2000,
    max = race_results_view['year'].max(),
    step = 1,
    value =  race_results_view['year'].max(),
    marks={str(year): str(year) for year in race_results_view['year'].unique()}
    ),
    dcc.Graph(id = "graph")
])

@app.callback(
    [Output("filter", "options"), Output("filter", "value")],
    Input("year_slider", "value"),
)
def update_rider_dropdown(selected_year):
    riders = race_results_view.loc[
        (race_results_view["year"] == selected_year) & (race_results_view["category"] == "MotoGP")
    ]["rider_name"].unique()
    rider_options = [{"label": rider, "value": rider} for rider in riders]
    riders_2021 = race_results_view.loc[(race_results_view['year'] == selected_year) & (race_results_view['category'] == 'MotoGP')]
    points_by_rider_sum = riders_2021.groupby('rider_name')['points'].sum().reset_index().sort_values('points', ascending = False)
    default_values = points_by_rider_sum.head(3)['rider_name'].tolist()
    return rider_options, default_values

@app.callback(
    Output("graph", "figure"),
    Input("filter","value"),
    Input("year_slider","value")
)
def plot_scatter(rider_names, year):
    
    riders_2021 = race_results_view.loc[(race_results_view['year'] == year) & (race_results_view['category'] == 'MotoGP')]
    points_by_race_and_rider = riders_2021.groupby(['sequence', 'rider_name'])['points'].sum().reset_index().sort_values('rider_name')
    dummy_points = pd.DataFrame({'rider_name': points_by_race_and_rider['rider_name'].unique(), 'sequence': 0, 'points': 0})
    points_by_race_and_rider = pd.concat([dummy_points, points_by_race_and_rider], ignore_index=True)
    grouped_points = points_by_race_and_rider.sort_values('sequence').groupby('rider_name')['points'].cumsum()
    points_by_race_and_rider['cumulative_points'] = grouped_points

    fig = px.line(points_by_race_and_rider.sort_values('sequence').query("rider_name in @rider_names"),
                  x = 'sequence',
                  y = 'cumulative_points',
                  color = 'rider_name',
                  markers=True,
                 category_orders={'rider_name': top_5_riders['rider_name']}
    )
    fig.update_xaxes(
        title = 'Number of round'
    )
    fig.update_yaxes(
        title = 'Total points'
    )
    fig.update_layout(
        title = 'Riders points gain in season 2021',
        title_x = 0.5,
        title_font_size = 20,
        legend_title = 'Rider name',
        yaxis=dict(rangemode='tozero'),
        xaxis=dict(rangemode='tozero')

    )

    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", port = 8111)

Dash is running on http://127.0.0.1:8111/



### Percent of finished races for riders that have started over 50 races

In [3]:
#BOX percent of finished races, riders with more than 50 starts
rider_stats = rider_stats.sort_values('starts', ascending = False)
#rider_stats.head()
rider_stats['finishes_per_start'] = 100 * rider_stats['finishes'] / rider_stats['starts']
rider_stats['rider_full_name'] = rider_stats['first_name'] + ' ' + rider_stats['last_name']

fig_box = px.box(rider_stats[rider_stats['starts'] > 50], 
                 y = 'finishes_per_start', 
                 hover_data = ['rider_full_name'], 
                 points="all")

fig_box.update_layout(
    title = 'Percent of finished races, riders with more than 50 starts',
    title_x = 0.5,
    title_font_size = 20,
)
fig_box.update_yaxes(
    title = 'Percent of finished races'
)

### Number of started races shown on box plot

In [4]:
# BOX amount of started races
fig_box_starts = px.box(rider_stats[rider_stats['starts'] >0], y = 'starts', hover_data = ['rider_full_name'])

fig_box_starts.update_layout(
    title = 'Amount of started races',
    title_x = 0.5,
    title_font_size = 20,
)
fig_box_starts.update_yaxes(
    title = 'Amount of races'
)
fig_box_starts.show()

### Top 20 riders with the most races started

In [5]:
#Top 20 riders with the most races started BAR
top_30_started_riders = rider_stats.head(30).sort_values('starts')

fig_bar_top_starts = px.bar(top_30_started_riders.sort_values('starts', ascending = False), x = 'rider_full_name', y = 'starts', color = 'starts')

fig_bar_top_starts.update_layout(
    title = 'Top 20 riders with the most races started',
    title_x = 0.5,
    title_font_size = 20,
)
fig_bar_top_starts.update_yaxes(
    title = 'Amount of races'
)
fig_bar_top_starts.update_xaxes(
    title = 'Rider'
)
fig_bar_top_starts.show()

### Top 20 tracks with most races in history

In [6]:
df_grouped = race_results_view.groupby(['year', 'sequence']).first()
circuit_counts = df_grouped['circuit_name'].value_counts()
circuit_counts_df = circuit_counts.reset_index()
circuit_counts_df.rename(columns={'index': 'circuit',
                                 'circuit_name': 'amount'}, inplace=True)

circuits_bar = px.bar(circuit_counts_df.head(20), x = 'count', y = 'amount', color = 'amount')

circuits_bar.update_layout(
    title = 'Top 20 tracks with most races in history',
    title_x = 0.5,
    title_font_size = 20,
)
circuits_bar.update_yaxes(
    title = 'Amount of races'
)
circuits_bar.update_xaxes(
    title = 'Track name'
)
circuits_bar.show()

### Riders performance on wet and dry track

In [7]:
rider_stats_2005_gp = race_results_view[(race_results_view['year'] >= 2005) & (race_results_view['category'] == "MotoGP")]
rider_stats_2005_gp = rider_stats_2005_gp.loc[:, ['year', 'sequence','rider_name','points']]

motoGP_conditions = race_condition[(race_condition['category'] == "MotoGP")]
motoGP_conditions = motoGP_conditions.loc[:, ['year','sequence','track']]

merged_df = rider_stats_2005_gp.merge(motoGP_conditions[['year', 'sequence', 'track']], on=['year', 'sequence'])


app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id = "filter",
        options = merged_df['rider_name'].unique(),
        value = "Rossi, Valentino"
    ),
    dcc.Graph(id = "graph")
])

@app.callback(
    Output("graph", "figure"),
    Input("filter","value")
)
def plot_box(rider_name):
    dry_track_results = merged_df[(merged_df['track'] == 'Dry') & (merged_df['rider_name'] == rider_name)]
    dry_track_results_sum = dry_track_results.groupby(['year', 'sequence', 'rider_name'])['points'].sum().reset_index()

    wet_track_results = merged_df[(merged_df['track'] == 'Wet') & (merged_df['rider_name'] == rider_name)]
    wet_track_results_sum = wet_track_results.groupby(['year', 'sequence', 'rider_name'])['points'].sum().reset_index()

    fig = go.Figure()
    fig.add_trace(go.Box(y = dry_track_results_sum['points'],name = 'Dry track', marker_color = 'indianred'))
    fig.add_trace(go.Box(y = wet_track_results_sum['points'], name = 'Wet track', marker_color = 'royalblue'))
    
    fig.update_layout(
        title = 'Rider points on dry and wet track',
        title_x = 0.5,
        title_font_size = 20
    )
    fig.update_yaxes(
        title = 'Points'
    )

    return fig

if __name__ == "__main__":
    app.run_server(mode="inline", port = 8112)


Dash is running on http://127.0.0.1:8112/



### How many races have held motoGP ivents in each country, MAP

In [8]:
merged_countries1 = race_results_view.merge(circuits[["name","short_name","country"]],left_on= 'circuit_name', right_on= 'short_name')
df_grouped = merged_countries1.groupby(['year', 'sequence']).first()
country_counts = df_grouped['country_y'].value_counts()
country_counts_df = country_counts.reset_index()
country_counts_df.rename(columns={'country_y': 'iso_code'}, inplace=True)

merged_countries = country_counts_df.merge(countries, on="iso_code", how = 'left')


fig = px.choropleth(merged_countries,
                    locations="gp_code",
                    color = "count",
                    labels = {"country_y" : "Number of races"}
                   )

fig.update_layout(
    title_text = 'Number of races in each country',
    title_x = 0.5,
    title_font_size = 20
)

fig.show()